# Cities and demographics data

Importing libraries

In [124]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata
import re
import pymysql

All relevant German cities/towns near an airport with a commercial flight scheduled

In [27]:
cities = ['Berlin', 'Borkum', 'Bremen','Cologne','Dortmund', 'Dresden', 'Düsseldorf', 'Emden', 'Erfurt', 'Frankfurt', 'Friedrichshafen', 'Hamburg', 'Hannover', 'Heide', 'Helgoland', 'Heringsdorf','Karlsruhe','Kassel', 'Koblenz','Leipzig', 'Halle (Saale)', 'Mannheim', 'Memmingen', 'Munich', 'Osnabrück','Nuremberg', 'Paderborn', 'Rostock', 'Saarbrücken', 'Stuttgart' , 'Weeze' ]



def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict

list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities_demographic = pd.DataFrame(list_of_city_info)



In [28]:
df_cities_demographic.isna().sum()

city                 0
mayor               19
city_size           14
elevation            3
city_population      0
urban_population    20
metro_population    14
lat                  0
long                 0
dtype: int64

Dropping cloumns with too many NaNs

In [29]:
df_cities_demographic2 = df_cities_demographic.drop(columns=['mayor', 'urban_population', 'metro_population', 'city_size']).copy()

In [151]:
df_cities_demographic2 = (df_cities_demographic2
                 .assign(country = lambda x: 'DE')
                 .assign(munic_country = lambda x: x['city']+','+x['country'])
                 
                )
df_cities_demographic2

,city,elevation,city_population,lat,long,country,munic_country
0,Berlin,34 m (112 ft),"3,769,495",52°31′12″N,13°24′18″E,DE,"Berlin,DE"
1,Borkum,6 m (20 ft),"5,002",53°35′17″N,06°40′11″E,DE,"Borkum,DE"
2,Bremen,12 m (39 ft),"566,573",53°5′N,8°48′E,DE,"Bremen,DE"
3,Cologne,37 m (121 ft),"1,083,498",50°56′11″N,6°57′10″E,DE,"Cologne,DE"
4,Dortmund,86 m (282 ft),"587,696",51°31′N,7°28′E,DE,"Dortmund,DE"
5,Dresden,113 m (371 ft),"556,227",51°03′00″N,13°44′24″E,DE,"Dresden,DE"
6,Düsseldorf,38 m (125 ft),"620,523",51°14′N,6°47′E,DE,"Düsseldorf,DE"
7,Emden,1 m (3 ft),"49,874",53°22′1″N,07°12′22″E,DE,"Emden,DE"
8,Erfurt,194 m (636 ft),"213,692",50°59′0″N,11°2′0″E,DE,"Erfurt,DE"
9,Frankfurt,112 m (367 ft),"764,104",50°7′N,8°41′E,DE,"Frankfurt,DE"


Creating the main cities table to link up

In [100]:
df_cities_GER = df_cities_demographic2.filter(['city']).copy()

In [102]:
df_cities_GER = (df_cities_GER
                 .assign(country = lambda x: 'DE')
                 .assign(munic_country = lambda x: x['city']+','+x['country'])
                 
                )

In [103]:
df_cities_GER

,city,country,munic_country
0,Berlin,DE,"Berlin,DE"
1,Borkum,DE,"Borkum,DE"
2,Bremen,DE,"Bremen,DE"
3,Cologne,DE,"Cologne,DE"
4,Dortmund,DE,"Dortmund,DE"
5,Dresden,DE,"Dresden,DE"
6,Düsseldorf,DE,"Düsseldorf,DE"
7,Emden,DE,"Emden,DE"
8,Erfurt,DE,"Erfurt,DE"
9,Frankfurt,DE,"Frankfurt,DE"


# Airport data

Importing airport data from csv

In [136]:
df_all_airports = pd.read_csv('airports.csv')

In [137]:
df_all_airports['type'].unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

Filtering out everything that isn't a normal airport

In [138]:
df_all_airports_filter = df_all_airports[df_all_airports['type'].str.contains("airport")]

Getting the German airports

In [139]:
df_airports_ger_all = (df_all_airports_filter
                   .query('iso_country == "DE"')
                   .filter(['name','municipality','iso_country','iso_region','gps_code','iata_code','latitude_deg','longitude_deg'])
                   .rename(columns={'iso_country':'country', 'iso_region':'region', 'gps_code':'icao_code', 'latitude_deg':'lat', 'longitude_deg': 'long'})
                   .assign(munic_country = lambda x: x['municipality']+','+ x['country'])
                  )

In [140]:
df_airports_ger_all.isna().sum()

name               0
municipality     119
country            0
region             0
icao_code        359
iata_code        745
lat                0
long               0
munic_country    119
dtype: int64

In [141]:
df_airports_ger_all.dropna(subset =['icao_code', 'municipality'], inplace=True)

In [143]:
df_airports_ger_all.isna().sum()

name               0
municipality       0
country            0
region             0
icao_code          0
iata_code        363
lat                0
long               0
munic_country      0
dtype: int64

In [144]:
df_airports_ger_all.reset_index(drop=True)

,name,municipality,country,region,icao_code,iata_code,lat,long,munic_country
0,Flugplatz Gransee,Gransee,DE,DE-BR,EDUG,NaN,53.006699,13.205000,"Gransee,DE"
1,Wilsche Glider Field,Wilsche,DE,DE-NI,EDVQ,NaN,52.524677,10.462597,"Wilsche,DE"
2,Wächtersberg-Hub Glider Airfield,Wildberg,DE,DE-BW,EDSV,NaN,48.614931,8.755457,"Wildberg,DE"
3,Brockzetel Glider Field,Aurich,DE,DE-NI,ETJA,NaN,53.481400,7.651100,"Aurich,DE"
4,Blexen Airfield,Blexen,DE,DE-NI,EDWT,NaN,53.538300,8.538800,"Blexen,DE"
...,...,...,...,...,...,...,...,...,...
437,Schleswig Air Base,Jagel,DE,DE-SH,ETNS,WBG,54.459301,9.516330,"Jagel,DE"
438,Wiesbaden Army Airfield,Wiesbaden,DE,DE-HE,ETOU,WIE,50.049801,8.325400,"Wiesbaden,DE"
439,Ingolstadt Manching Airport,Manching,DE,DE-BY,ETSI,IGS,48.715698,11.534000,"Manching,DE"
440,Lechfeld Air Base,Lagerlechfeld,DE,DE-BY,ETSL,NaN,48.185504,10.861200,"Lagerlechfeld,DE"


Getting all the relevant German airports for our approach

In [145]:
df_airports_ger_final = df_cities_GER.merge(df_airports_ger_all, on='munic_country', how='inner').copy()

In [146]:
df_airports_ger_final.drop(columns=['city', 'country_x'], inplace=True)   

In [147]:
df_airports_ger_final.rename(columns = {'country_y' : 'country'}, inplace=True)

In [148]:
df_airports_ger_final

,munic_country,name,municipality,country,region,icao_code,iata_code,lat,long
0,"Berlin,DE",Berlin Brandenburg Airport,Berlin,DE,DE-BR,EDDB,BER,52.351389,13.493889
1,"Borkum,DE",Borkum Airport,Borkum,DE,DE-NI,EDWR,BMK,53.596390,6.709167
2,"Bremen,DE",Bremen Airport,Bremen,DE,DE-HB,EDDW,BRE,53.047501,8.786670
3,"Cologne,DE",Cologne Bonn Airport,Cologne,DE,DE-NW,EDDK,CGN,50.865898,7.142740
4,"Dortmund,DE",Dortmund Airport,Dortmund,DE,DE-NW,EDLW,DTM,51.518299,7.612240
5,"Dresden,DE",Dresden Airport,Dresden,DE,DE-SN,EDDC,DRS,51.132801,13.767200
6,"Düsseldorf,DE",Düsseldorf Airport,Düsseldorf,DE,DE-NW,EDDL,DUS,51.289501,6.766780
7,"Emden,DE",Emden Airport,Emden,DE,DE-NI,EDWE,EME,53.391109,7.227500
8,"Erfurt,DE",Erfurt Airport,Erfurt,DE,DE-TH,EDDE,ERF,50.979801,10.958100
9,"Friedrichshafen,DE",Friedrichshafen Airport,Friedrichshafen,DE,DE-BW,EDNY,FDH,47.671299,9.511490


# Upload to cloud database

setting up the connection

In [149]:
user=user_id
password=password
host=endpoint
database=db_name
port=port
con= f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

uploading the 3 tables

In [128]:
df_cities_GER.to_sql('cities', con=con, index=False)

In [150]:
df_airports_ger_final.to_sql('airports_ger', con=con, index=False)

In [152]:
df_cities_demographic2.to_sql('cities_demographics', con=con, index=False)